In [1]:
import sys
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd

In [3]:
# df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/NonPromoterSequence.txt', sep = '>', )
# df.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
# df.reset_index(inplace = True)
# df.drop(['EP 1 (+) mt:CoI_1; range -400 to -100.', 'index'], axis = 1, inplace=True) #data cleaning after error found
# df.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
# df['label'] = 0

# df2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/PromoterSequence.txt', sep = '>', )
# df2.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
# df2.reset_index(inplace = True)
# df2.drop(['EP 1 (+) mt:CoI_1; range -100 to 200.', 'index'], axis = 1, inplace=True)
# df2.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
# df2['label'] = 1

# df = pd.concat([df, df2], axis = 0 )

In [4]:
# for seq in df['sequence']:
#     if 'N' in seq:
#         display(df.loc[df['sequence'] == seq])

# df.drop([1822], inplace = True)

In [5]:

# sequence = list(df.loc[:, 'sequence'])
# encoded_seq = []

# def seq2kmer(seq, k):
#     """
#     Convert original sequence to kmers
    
#     Arguments:
#     seq -- str, original sequence.
#     k -- int, kmer of length k specified.
    
#     Returns:
#     kmers -- str, kmers separated by space
#     """
#     kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
#     kmers = " ".join(kmer)
#     return kmers

# for seq in sequence:
#   x = seq2kmer(seq, 6)
#   encoded_seq.append(x)

# print(encoded_seq[0])


In [6]:
# label = list(df.loc[:,'label'])
# new_df = pd.DataFrame(encoded_seq, columns = ['sequence'])

# new_df.insert(1, 'label', label)

# new_df.head()

In [7]:
# new_df.to_csv("dnabert_input.csv", mode='a', header=False)

In [8]:
!pip install transformers

import numpy as np
import os

import joblib
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import utils
import torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.2 MB/s 
     |████████████████████████████████| 182 kB 76.2 MB/s 
     |████████████████████████████████| 7.6 MB 53.4 MB/s 


In [9]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/dnabert_input.csv', sep = ',', )
df.columns = ['id', 'sequence', 'label']
df.drop(columns = ['id'])


,sequence,label
0,ATTTTT TTTTTA TTTTAC TTTACA TTACAA TACAAG ACAA...,0
1,AGAGAT GAGATA AGATAG GATAGG ATAGGT TAGGTG AGGT...,0
2,TATGTA ATGTAT TGTATA GTATAT TATATA ATATAG TATA...,0
3,AGAAAT GAAATA AAATAA AATAAT ATAATA TAATAG AATA...,0
4,CTTTAA TTTAAA TTAAAA TAAAAT AAAATT AAATTA AATT...,0
...,...,...
22592,CGACAA GACAAA ACAAAG CAAAGT AAAGTT AAGTTT AGTT...,1
22593,CATATC ATATCT TATCTA ATCTAC TCTACA CTACAT TACA...,1
22594,ATACCG TACCGC ACCGCG CCGCGG CGCGGA GCGGAA CGGA...,1
22595,ATTATT TTATTC TATTCC ATTCCG TTCCGA TCCGAA CCGA...,1


In [10]:
df.shape

(22597, 3)

In [11]:
# Import the necessary 
!pip install --upgrade transformers

import transformers
from transformers import AutoModel, AutoTokenizer
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.numpy_ops import np_config
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

# Load your dataset

data = df

# data['sequence'] = input_sequences

# data.tail()

# Divide the dataset into train, validation, and test sets
train_data, val_data = train_test_split(
    data, 
    test_size=0.1, 
    random_state=42,
    stratify=data['label']
)
val_data, test_data = train_test_split(
    train_data, 
    test_size=0.5, 
    random_state=42,
    stratify=train_data['label']
)

device = torch.device('cuda')
use_cuda = True

train_input = tokenizer.batch_encode_plus(train_data["sequence"].tolist(), add_special_tokens=True)['input_ids']
val_input = tokenizer.batch_encode_plus(val_data["sequence"].tolist(), add_special_tokens=True)['input_ids']
test_input = tokenizer.batch_encode_plus(test_data["sequence"].tolist(), add_special_tokens=True)['input_ids']

train_input = torch.tensor(train_input).to(device)
train_labels = torch.tensor(train_data['label'].values).to(device)

val_input = torch.tensor(val_input).to(device)
val_labels = torch.tensor(val_data['label'].values).to(device)

test_input = torch.tensor(test_input).to(device)
test_labels = torch.tensor(test_data['label'].values).to(device)

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [15]:
model = AutoModel.from_pretrained("armheb/DNA_bert_6")

finetuning_layer_1 = torch.nn.Linear(768, 256)

finetuning_layer_2 = torch.nn.Linear(256, 1)

# Combine the BERT model and the finetuning layers
model = torch.nn.Sequential(model, finetuning_layer_1, finetuning_layer_2)

if use_cuda and torch.cuda.is_available():
    model.cuda()

# Define the optimizer, loss function, and number of training epochs
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = nn.BCEWithLogitsLoss()
epochs = 10
batch_size = 32


Some weights of the model checkpoint at armheb/DNA_bert_6 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
loss_history = {"train": [], "val":[]}

train_input_size = len(train_input)

for epoch in range(epochs):
    # Set the model to training mode
    model.train()

    epoch_loss = 0.0

    # Loop over each batch of training data
    for i in range(0, len(train_input), batch_size):
        # Get the current batch of data
        batch_input_ids = train_input[i:i+batch_size]
        batch_labels = train_labels[i:i+batch_size]
        # Forward pass
        outputs = model(batch_input_ids)

        # Compute the loss
        loss = loss_fn(outputs, batch_labels)
        epoch_loss += loss

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()
        optimizer.zero_grad()

    epoch_loss = epoch_loss / train_input_size
    loss_history['train'].append(epoch_loss)
    # Set the model to evaluation mode
    model.eval()

    # Evaluate the model on the validation set
    with torch.no_grad():
        # Forward pass
        outputs = model(val_input)

        # Compute the loss
        loss = loss_fn(outputs, val_labels)
        loss_history['val'].apend(epoch_loss)
        # Print the validation loss
        print(f'Epoch {epoch+1}/{epochs}: train loss {epoch_loss:.4f}, validation loss {loss:.4f}')

# Save the finetuned model
torch.save(model.state_dict(), 'finetuned_model.pth')

TypeError: ignored

In [ ]:
from torchvision import utils
import matplotlib.pyplot as plt

plt.title("Train-Val Loss")
plt.plot(range(1,epochs+1),loss_history["train"], label='train')
plt.plot(range(1,epochs+1),loss_history["val"], label='train')
plt.ylabel("Loss")
plt.xlabel("Training Epochs")
plt.legend()
plt.show()
